In [1]:
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

## SMO algorithm

In [2]:
def loadDataSet(fileName):
    fr = open(fileName)
    dataList = []
    labelList = []
    for line in fr.readlines():
        lineList = line.strip().split('\t')
        vec = [float(x) for x in lineList[:-1]]
        dataList.append(vec)
        label = float(lineList[-1])
        labelList.append(label)
        
    return dataList, labelList

def selectJrand(i, m):
    # i : current indexs
    # 0~m : all indexs
    j = i
    while (j == i):
        j = int(np.random.uniform(0,m))
    return j

def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if aj < L:
        aj = L
        
    return aj

In [3]:
dataList, labelList = loadDataSet("data/svmTest/testSet.txt")
print("***** data abstract ***")
print(dataList[:10])
print(np.shape(dataList))
print(labelList[:10])
print(np.shape(labelList))

***** data abstract ***
[[3.542485, 1.977398], [3.018896, 2.556416], [7.55151, -1.58003], [2.114999, -0.004466], [8.127113, 1.274372], [7.108772, -0.986906], [8.610639, 2.046708], [2.326297, 0.265213], [3.634009, 1.730537], [0.341367, -0.894998]]
(100, 2)
[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0]
(100,)


## SMO symple version  

**algorithm**  
```code
create an 'alpha' vector and initialize to 0-vector  
when number of iteration less than maximun iteration:  
    for each instance 'ins' in dataset:  
        if 'ins' can be optimized:  
            randomly select another instance 'ins2'    
            optimize 'ins' and 'ins2' simultaneously  
            if two of these instance can't be optimized, then exit current loop  
    if all of instances haven't been optimized, then increasing the number of iteration and continue next loop  
```

In [4]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    b = 0; m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])#if checks if an example violates KKT conditions
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print "L==H"; continue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0: print "eta>=0"; continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                        #the update is in the oppostie direction
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print "iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print "iteration number: %d" % iter
    return b,alphas

In [6]:
# numpy.multiply : element-wise  
# numpy.dot : matrix multiplication

In [7]:
test = [[1,2,3],[3,4,5]]
test

[[1, 2, 3], [3, 4, 5]]

In [7]:
np.shape(test)

(2, 3)

In [9]:
np.shape(np.mat(test))

(2, 3)

In [12]:
np.shape(np.mat(labelList).transpose())

(100, 1)